<a href="https://colab.research.google.com/github/kmkarakaya/Deep-Learning-Tutorials/blob/master/Simple_Rag_with_chromaDB_Gemini_PartC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART C: CODE WITH CHROMADB FOR PERSISTENT VECTOR DB   
    
    
    
    
    

In this notebook we will develop a Retrieval Augmented Generation (RAG) application.

The Parts are

* PART A: AN INTRO TO GEMINI API FOR TEXT GENERATION & CHAT
* PART B: CODE WITH CHROMADB FOR VECTOR STORAGE & SIMILARITY SEARCH
* PART C: CODE WITH CHROMADB FOR PERSISTENT VECTOR DB
* PART D: A SIMPLE RAG BASED ON GEMINI & CHROMADB
* PART E: ADVANCED TECHNIQUES FOR RAG BASED ON GEMINI & CHROMADB

Watch this notebook on the Murat Karakaya Akademi YouTube channel:
* In English: https://www.youtube.com/playlist?list=PLQflnv_s49v-EFKdOVDKB743f1iskLLw2
* In Turkish: https://www.youtube.com/playlist?list=PLQflnv_s49v_nrk7iGYqw5iRAKrSZPnnV

# WHAT IS RAG?

RAG stands for Retrieval-Augmented Generation. It's a technique that combines large language models (LLMs) with external knowledge sources to improve the accuracy and reliability of AI-generated text.

## How Does RAG Work? Unveiling the Power of External Knowledge

Before we start the core RAG process, we need to provide a foundation as follows:

* **Building the Knowledge Base:** The system starts by transforming documents and information within the external knowledge base (like Wikipedia or a company database) into a special format called **vector representations**. These condense the meaning of each document into a series of **numbers**, capturing the essence of the content.

* **Vector Database for Speedy Retrieval**: These vector representations are then stored in a specialized database called a vector database. This database is optimized for efficiently **searching and retrieving** information based on **semantic similarity**. Imagine it as a super-powered library catalog that **understands the meaning** of documents, **not just keywords**.

Now, let's explore how RAG leverages this foundation:

* **User Input**: The RAG process begins with a question or **prompt** from the user. This could be anything from "What caused the extinction of the dinosaurs?" to a more open-ended request like "Write a creative story."

* **Intelligent Retrieval**: RAG doesn't rely solely on the **LLM's internal knowledge**. It employs an information retrieval component that acts like a super-powered search engine. This component scans the vast external knowledge base – like a company's internal database for specific domains – to find information **directly relevant** to the user's input. Unlike a traditional **search engine** that relies on **keywords**, RAG leverages the power of vector representations to understand the **semantic meaning** of the user's prompt and identify the most relevant documents.

* **Enriched Context Creation**: The retrieved information isn't just shown alongside the prompt. RAG cleverly **merges the user input with the relevant snippets** from the knowledge base. This creates a ***richer context*** for the LLM to understand the **user's intent** and formulate a well-informed response.

* **LLM Powered Response Generation**: Finally, the **enriched context** is fed to the Large Language Model (LLM). The LLM, along with its ability to process language patterns, now has a strong **foundation of factual** information to draw upon. This empowers it to generate a response that is both comprehensive and accurate, addressing the specific needs of the user's prompt.

In this part, we will learn how to build a persistent ChromaDB Vector Database for speedy retrieval in a Knowledge Base.

https://www.trychroma.com/
https://github.com/chroma-core/chroma

# CONTENT

In this exciting tutorial series, we are developing a Retrieval Augmented Generation (RAG) application. If you missed the first 2 parts where we covered how to code the GEMINI API for text generation and chat and how to code ChromaDB to store and retrieve vectors, be sure to check that out.

In this second part, we will code with ChromaDB for a persistent Database.

In this tutorial, we will learn:

🔍 How Does RAG Work?

🛠️ Create a Persistent ChromaDB Client

📄 Upload Single and Multiple PDF Documents to ChromaDB

💾 Save the ChromaDB Collection

🔄 Reload the ChromaDB Client and Collection

All the above steps will be implemented and coded in Python on Google Colab.

Follow along step-by-step to master these techniques and enhance your data processing capabilities.

# WHY WE NEED A PERSISTENT CHROMADB?

In the context of a Retrieval-Augmented Generation (RAG) approach, saving and loading a persistent ChromaDB is particularly important for several reasons:

1. **Enhanced Data Durability**:
   - **Importance**: Ensures the retrieval database used for augmenting generative models is not lost between sessions or system restarts.
   - **RAG Relevance**: Maintains a consistent and reliable knowledge base that the generative model can reference, leading to more accurate and relevant responses.

2. **Operational Continuity**:
   - **Importance**: Allows seamless continuation of operations without needing to re-index or re-import data, saving time and computational resources.
   - **RAG Relevance**: Ensures that the generative model has continuous access to the same set of documents, which is essential for generating consistent and coherent responses over time.

3. **Facilitating Collaboration**:
   - **Importance**: Enables multiple users or systems to share and access the same dataset.
   - **RAG Relevance**: Supports collaborative development and usage of the RAG system, allowing different teams to work on improving the retrieval and generation processes simultaneously.

4. **Scalability**:
   - **Importance**: Provides a stable and persistent backend, enabling efficient handling of large datasets.
   - **RAG Relevance**: Essential for scaling the RAG system to handle more extensive and diverse knowledge bases, ensuring that the system can manage increased loads and deliver prompt, relevant information.


In a RAG system, the retriever (like ChromaDB) provides the generative model with relevant context from a knowledge base to generate informed and accurate responses. Persistent storage ensures that this knowledge base is durable, continuously available, and scalable, which is critical for the reliability, consistency, and performance of the RAG system.



# CREATING & SAVING A PERSISTENT CHROMADB

To make ChromaDB durable (persistent) rather than temporary on Google Colab, you can use external storage services like Google Drive or set up a cloud-based database. Google Colab provides temporary storage that resets after each session, so to maintain persistence across sessions, you'll need to save your data and configurations externally.

##1 Install required libraries

Install all the required libraries and helper functions

In [ ]:
%pip install chromadb --quiet
%pip install sentence_transformers --quiet
%pip install pypdf --quiet
%pip install langchain --quiet
%pip install tqdm --quiet

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter


from pypdf import PdfReader

from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient
from chromadb.utils import embedding_functions

import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## 1. Initialize ChromaDB client with Google Drive connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# Initialize ChromaDB client with Google Drive connection
chromaDB_path = '/content/drive/MyDrive/Colab Notebooks/ChromaDBData'


In [ ]:
# Check if the chromadb_path exists or not. If so, delete all the files and folders in chromadb_path. But before deleting get the permission from the user.

import os
import shutil

def delete_all_files_and_folders(chromaDB_path):
  if os.path.exists(chromaDB_path):
    print(f"The directory '{chromaDB_path}' already exists.")
    permission = input("Do you want to delete all the files and folders in this directory? (y/n): ")
    if permission == "y":
      shutil.rmtree(chromaDB_path)
      print(f"All files and folders in '{chromaDB_path}' have been deleted.")
    else:
      print("No action taken.")
  else:
    print(f"The directory '{chromaDB_path}' does not exist.")



In [ ]:
delete_all_files_and_folders(chromaDB_path)

The directory '/content/drive/MyDrive/Colab Notebooks/ChromaDBData' already exists.
Do you want to delete all the files and folders in this directory? (y/n): y
All files and folders in '/content/drive/MyDrive/Colab Notebooks/ChromaDBData' have been deleted.


## 2. Define PersistentClient

Let's re-define the **create_chroma_client** function from the previous part so that this time we initialize a **persistent** ChromaDB client:

In [ ]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient


In [ ]:
def create_chroma_client(collection_name, embedding_function, chromaDB_path ):
  if chromaDB_path is not None:
    chroma_client = PersistentClient(path=chromaDB_path,
                                     settings=Settings(),
                                     tenant=DEFAULT_TENANT,
                                     database=DEFAULT_DATABASE,)
  else:
    chroma_client = Client()

  chroma_collection = chroma_client.get_or_create_collection(
      collection_name,
      embedding_function=embedding_function)

  return chroma_client, chroma_collection

## 3. Create a collection as usual

In [ ]:
collection_name = "Papers"
sentence_transformer_model="distiluse-base-multilingual-cased-v1"
embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
chroma_client, chroma_collection = create_chroma_client(collection_name,
                                                        embedding_function,
                                                        chromaDB_path)
print(chroma_collection.count())
print(chroma_client.list_collections())

0


## Define helper functions

In [ ]:
from google.colab import files
def upload_multiple_files():
  uploaded = files.upload()
  file_names = list()
  for fn in uploaded.keys():
    #print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    file_names.append(fn)
  return file_names

In [ ]:
def convert_PDF_Text(pdf_path):
  reader = PdfReader(pdf_path)
  pdf_texts = [p.extract_text().strip() for p in reader.pages]
  # Filter the empty strings
  pdf_texts = [text for text in pdf_texts if text]
  print("Document: ",pdf_path," chunk size: ", len(pdf_texts))
  return pdf_texts

In [ ]:
def convert_Page_ChunkinChar(pdf_texts, chunk_size = 1500, chunk_overlap=0 ):
  character_splitter = RecursiveCharacterTextSplitter(
      separators=["\n\n", "\n", ". ", " ", ""],
      chunk_size=1500,
      chunk_overlap=0
)
  character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))
  print(f"\nTotal number of chunks (document splited by max char = 1500): \
        {len(character_split_texts)}")
  return character_split_texts

In [ ]:
def convert_Chunk_Token(text_chunksinChar,sentence_transformer_model, chunk_overlap=0,tokens_per_chunk=128 ):
  token_splitter = SentenceTransformersTokenTextSplitter(
      chunk_overlap=0,
      model_name=sentence_transformer_model,
      tokens_per_chunk=128)

  text_chunksinTokens = []
  for text in text_chunksinChar:
      text_chunksinTokens += token_splitter.split_text(text)
  print(f"\nTotal number of chunks (document splited by 128 tokens per chunk):\
       {len(text_chunksinTokens)}")
  return text_chunksinTokens

In [ ]:
def add_meta_data(text_chunksinTokens, title, category, initial_id):
  ids = [str(i+initial_id) for i in range(len(text_chunksinTokens))]
  metadata = {
      'document': title,
      'category': category
  }
  metadatas = [ metadata for i in range(len(text_chunksinTokens))]
  return ids, metadatas

In [ ]:
def add_document_to_collection(ids, metadatas, text_chunksinTokens, chroma_collection):
  print("Before inserting, the size of the collection: ", chroma_collection.count())
  chroma_collection.add(ids=ids, metadatas= metadatas, documents=text_chunksinTokens)
  print("After inserting, the size of the collection: ", chroma_collection.count())
  return chroma_collection

In [ ]:
def retrieveDocs(chroma_collection, query, n_results=5, return_only_docs=False):
    results = chroma_collection.query(query_texts=[query],
                                      include= [ "documents","metadatas",'distances' ],
                                      n_results=n_results)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results

In [ ]:
def show_results(results, return_only_docs=False):

  if return_only_docs:
    retrieved_documents = results
    if len(retrieved_documents) == 0:
      print("No results found.")
      return
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print("\tDocument Text: ")
      display(to_markdown(doc));
  else:

      retrieved_documents = results['documents'][0]
      if len(retrieved_documents) == 0:
          print("No results found.")
          return
      retrieved_documents_metadata = results['metadatas'][0]
      retrieved_documents_distances = results['distances'][0]
      print("------- retreived documents -------\n")

      for i, doc in enumerate(retrieved_documents):
          print(f"Document {i+1}:")
          print("\tDocument Text: ")
          display(to_markdown(doc));
          print(f"\tDocument Source: {retrieved_documents_metadata[i]['document']}")
          print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
          print(f"\tDocument Distance: {retrieved_documents_distances[i]}")


## 4. Revise the load_multiple_pdfs_to_ChromaDB() to include persistentClient  

We need to update the function as well:

In [ ]:
def load_multiple_pdfs_to_ChromaDB(collection_name,sentence_transformer_model,
                                   chromaDB_path):

  collection_name= collection_name
  category= "Journal Paper"
  sentence_transformer_model=sentence_transformer_model
  embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)
  chroma_client, chroma_collection = create_chroma_client(collection_name, embedding_function, chromaDB_path)
  current_id = chroma_collection.count()
  file_names = upload_multiple_files()
  for file_name in file_names:
    print(f"Document: {file_name} is being processed to be added to the {chroma_collection.name} {chroma_collection.count()}")
    print(f"current_id: {current_id} ")
    pdf_texts = convert_PDF_Text(file_name)
    text_chunksinChar = convert_Page_ChunkinChar(pdf_texts)
    text_chunksinTokens = convert_Chunk_Token(text_chunksinChar,sentence_transformer_model)
    ids,metadatas = add_meta_data(text_chunksinTokens,file_name,category, current_id)
    current_id = current_id + len(text_chunksinTokens)
    chroma_collection = add_document_to_collection(ids, metadatas, text_chunksinTokens, chroma_collection)
    print(f"Document: {file_name} added to the collection: {chroma_collection.count()}")
  return  chroma_client, chroma_collection

## 5. Run load_multiple_pdfs_to_ChromaDB() to fill in the colection

In [ ]:
chroma_client, chroma_collection= load_multiple_pdfs_to_ChromaDB(collection_name,sentence_transformer_model, chromaDB_path)

Saving 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf to 15 UAV Route Planning For Maximum Target Coverage ABSTRACT (1).pdf
Saving 29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf to 29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT (2).pdf
Saving 30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf to 30 Time-Sensitive Ant Colony Optimization ORIGINAL (1).pdf
Saving 73 Analyzing Students Academic Success.pdf to 73 Analyzing Students Academic Success (1).pdf
Saving 77 Arac Park Yerlerinin Doluluk.pdf to 77 Arac Park Yerlerinin Doluluk (1).pdf
Document: 15 UAV Route Planning For Maximum Target Coverage ABSTRACT (1).pdf is being processed to be added to the Papers 0
current_id: 0 
Document:  15 UAV Route Planning For Maximum Target Coverage ABSTRACT (1).pdf  chunk size:  1

Total number of chunks (document splited by max char = 1500):         2

Total number of chu

## 6. Test the ChromaDB client and & collection

In [ ]:
query = "What are the main difference in active and passive path scheduling?"

'''
In 16 A Local Optimization Technique for Assigning New Targets ABSTRACT:

Route planning can be static or dynamic. In static route planning, routes are
constructed according to given UAVs and targets and do not change during
the mission. However, in dynamic route planning, number of routes or UAVs
can alter which requires the update of existing routes to adopt these changes.

'''

'\nIn 16 A Local Optimization Technique for Assigning New Targets ABSTRACT:\n\nRoute planning can be static or dynamic. In static route planning, routes are\nconstructed according to given UAVs and targets and do not change during\nthe mission. However, in dynamic route planning, number of routes or UAVs\ncan alter which requires the update of existing routes to adopt these changes.\n\n'

In [ ]:
retrieved_documents=retrieveDocs(chroma_collection, query, 10)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> impact of cirriculumdeveloment method. As seen above references, there have been a considerable interest on the pre - requisite coursesin the literature. However, the scope, goals, and results of the above - mentioned studies are not completely in line with those of the present work. One of the studies with similar scope and goals wasconducted by Anderson et al. [ 25 ]. They found thatsuccessfully completing calculus and economicscourses positively [UNK] the achieved success insubsequent economics courses. In another work, McMillan and Adeyemi focused on the success relationships between the

	Document Source: 73 Analyzing Students Academic Success (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.306624412536621
Document 2:
	Document Text: 


> ##lected data for a given period of time, namely tour time. Provided below are the details of the MES / TSACO implementation 3. 2 The Time - Sensitive ACO A combinatorial optimization problem can be static or dynamic with respect to the given characteristics of the problem. In static problems, the underly - ing system properties stay the same throughout the problem - solution pro - cess. A typical example of these kinds of problems is the Traveling Sales - man Problem ( TSP ). In the TSP, there are a number of towns ( vertices ) con - nected to each

	Document Source: 30 Time-Sensitive Ant Colony Optimization ORIGINAL (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.3876214027404785
Document 3:
	Document Text: 


...

	Document Source: 73 Analyzing Students Academic Success (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.4612693786621094
Document 9:
	Document Text: 


> In the proposed solution, each ant constructs routes for the given number of UAVs using pheromone and heuristic information. These routes are locally optimized using a modified 2 - opt technique. After each iteration, t he solution which covers more targets with less route distance is selected as the iteration - best solution and the pheromone values of the edges on that route are increased. According to the termination condition, the algorithm stops and outputs the best route found so far as the result. To evaluate the success of the proposed method, another approach, based on the Nearest Neigh

	Document Source: 15 UAV Route Planning For Maximum Target Coverage ABSTRACT (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.4616098403930664
Document 10:
	Document Text: 


> Second, the considered chainedcourses are limited to three mathematical courses ( Calculus I, Calculus II, and [UNK] Equa - tions ). Third, among the several curriculum devel - opment methods, only two ( Spiral and Linear ) areselected for this research. Lastly, since there are [UNK] records stating which courses were designed according to which curriculum development meth - odology, the [UNK] of the chains are doneaccording to the [UNK] of the cirriculum devel - opment methods. Murat Karakaya et al. 368

	Document Source: 73 Analyzing Students Academic Success (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.4657018184661865


## 7. Observe the ChromeDB saved to the provided path

List the folders and files in the chromaDB_path

In [ ]:
!ls "{chromaDB_path}"

9a5cf884-606f-478b-9aa4-a8288aeea09d  chroma.sqlite3


### YES WE DID IT!

# LOAD PERSISTENT CHROMADB



Let's kill the kernel so we ensure that nothing remains in the memory from all the above ChromaDB instance.

In [ ]:
from google.colab import runtime
# Disconnect from the runtime
#!kill -9 -1

##1 Connect to source directory

First get connected to the ChromaDB directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change directory to chromaDB folder
chromaDB_path = '/content/drive/MyDrive/Colab Notebooks/ChromaDBData'
%cd {chromaDB_path}


/content/drive/MyDrive/Colab Notebooks/ChromaDBData


### Check that if chromadb_path exist or not and if exists does it contain chromadb files and folders

In [ ]:
import os
if os.path.exists(chromaDB_path):
    print(f"The directory '{chromaDB_path}' exists.")

    # Check if the directory contains ChromaDB files and folders
    chromadb_files_and_folders = os.listdir(chromaDB_path)
    if any(file_or_folder.startswith('chroma') for file_or_folder in chromadb_files_and_folders):
        print("The directory contains ChromaDB files and folders.")
    else:
        print("The directory does not contain ChromaDB files and folders.")
else:
    print(f"The directory '{chromadb_path}' does not exist.")


The directory '/content/drive/MyDrive/Colab Notebooks/ChromaDBData' exists.
The directory contains ChromaDB files and folders.


##2 Install required libraries

Secondly install all the required libraries and helper functions

In [ ]:
%pip install chromadb --quiet
%pip install sentence_transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50

In [ ]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient
from chromadb.utils import embedding_functions


In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def retrieveDocs(chroma_collection, query, n_results=5,
                 return_only_docs=False, filterType=None, filterValue=None):
    if filterType is not None and filterValue is not None:
        results = chroma_collection.query(
            query_texts=[query],
            include=["documents", "metadatas", "distances"],
            where={filterType: filterValue},
            n_results=n_results)

    else:
        results = chroma_collection.query(
            query_texts=[query],
            include= [ "documents","metadatas",'distances' ],
            n_results=n_results)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results

In [ ]:
def show_results(results, return_only_docs=False):

  if return_only_docs:
    retrieved_documents = results
    if len(retrieved_documents) == 0:
      print("No results found.")
      return
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print("\tDocument Text: ")
      display(to_markdown(doc));
  else:

      retrieved_documents = results['documents'][0]
      if len(retrieved_documents) == 0:
          print("No results found.")
          return
      retrieved_documents_metadata = results['metadatas'][0]
      retrieved_documents_distances = results['distances'][0]
      print("------- retreived documents -------\n")

      for i, doc in enumerate(retrieved_documents):
          print(f"Document {i+1}:")
          print("\tDocument Text: ")
          display(to_markdown(doc));
          print(f"\tDocument Source: {retrieved_documents_metadata[i]['document']}")
          print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
          print(f"\tDocument Distance: {retrieved_documents_distances[i]}")


##3 Initailizing

 Now, we can begin to upload the persistent ChromaDB from the location by initailizing
*  the chromaDB client
*  the chromaDB collections

In [ ]:
# Initialize ChromaDB client with Google Drive connection
drive_path = '/content/drive/MyDrive/Colab Notebooks/ChromaDBData'

In [ ]:
chroma_client2 = PersistentClient(path=drive_path,
                                     settings=Settings(),
                                     tenant=DEFAULT_TENANT,
                                     database=DEFAULT_DATABASE,)

In [ ]:
for col in chroma_client2.list_collections():
  print (col.name)

Papers


In [ ]:
collection_name = "Papers"
sentence_transformer_model="distiluse-base-multilingual-cased-v1"
embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
chroma_collection2 = chroma_client2.get_or_create_collection(
      collection_name,
      embedding_function=embedding_function)

##4 Test

Test the loaded ChromeDB client and the collection

In [ ]:
chroma_collection2.get(['0'])

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [{'category': 'Journal Paper',
   'document': '15 UAV Route Planning For Maximum Target Coverage ABSTRACT (1).pdf'}],
 'documents': ['UAV Route Planning For Maximum Target Coverage Murat Karakaya Department of Computer Engineering, Atilim University Incek / Ankara TURKEY kmkarakaya @ atilim. edu. tr Abstract The importance and the impact of using Unmanned Aerial Vehicles ( UAVs ) in military and civil operations are increasing. One of the challenges in effectively tasking these expensive vehicles is planning the flight routes to monitor the targets. This problem is related with the Multiple Travelling Salesman Problem ( mTSP ) and the Vehicle Routing Problem ( VRP ). In these well - defined problem'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

```python
chroma_collection.get(['0'])

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [{'category': 'Journal Paper',
   'document': '15 UAV Route Planning For Maximum Target Coverage.pdf'}],
 'documents': ['Computer Science & Engineering : An International Journal ( CSEIJ ), Vol. 4, No. 1,
 February 2014 DOI : 10. 5121 / cseij. 2014. 410 3 27UAVROUTEPLANNING FORMAXIMUMTARGET COVERAGE MuratKarakaya1
 1Department of Computer Engineering, Atilim University, Ankara, Turkey ABSTRACT Utilization of Unmanned Aerial
 Vehicles ( UAVs ) in military and civil operations is getting popular. One of the challenges in effectively
  tasking these expensive vehicles is planning'],
 'uris': None,
 'data': None}
```

In [ ]:
query = "What is Target Coverage?"

In [ ]:
retrieved_documents=retrieveDocs(chroma_collection2, query, 10)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> s, it is mostly assumed that travelling salesm en or vehicles should visit all the targets and the target function is defined as to find a minimum - distan t route. Even, i n the constraint versions of the mTSP and VRP, some other restrictions ( visiting time windows, number of depots, etc. ) are included ; it is still assumed that there exists enough number of travelling salesm en or vehicle s to cover all the give n locations. However, in reality the number and flight range of UAVs might be insufficient to cover all the targets. As a result, t he maxim

	Document Source: 15 UAV Route Planning For Maximum Target Coverage ABSTRACT (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.2466243505477905
Document 2:
	Document Text: 


> impact of cirriculumdeveloment method. As seen above references, there have been a considerable interest on the pre - requisite coursesin the literature. However, the scope, goals, and results of the above - mentioned studies are not completely in line with those of the present work. One of the studies with similar scope and goals wasconducted by Anderson et al. [ 25 ]. They found thatsuccessfully completing calculus and economicscourses positively [UNK] the achieved success insubsequent economics courses. In another work, McMillan and Adeyemi focused on the success relationships between the

	Document Source: 73 Analyzing Students Academic Success (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.3942387104034424
Document 3:
	Document Text: 


...

	Document Source: 73 Analyzing Students Academic Success (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.5144762992858887
Document 9:
	Document Text: 


> lliklerini ezberleyeceği ( overfitting ) için başarı oranı gerçeği yansıtmayacak kadar yüksek çıkabilecektir. B irbirinden farklı görüntüler den oluşan ü ç veri setinin kullanımı ile D ESA [UNK] nın sınıflandırma başarı m ı hakkında daha gerçekçi sonuçlar elde edilmek i stenmiştir. Şekil 2 : Veri s etlerinde b ulunan p ark y eri g örüntülerinden ö rnekler B. Derin Evrişimsel Sinir Ağları Bu çalışma

	Document Source: 77 Arac Park Yerlerinin Doluluk (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.514948129310384
Document 10:
	Document Text: 


> ##quencing of topics, all approaches basically depend on the pre - requisite chains to implement their educational approach in the curriculum development for specifying the order of the subjects. In this research, two pre - requisite chains representing two [UNK] curriculum development approaches are taken into consideration in a casestudy. The [UNK] research question considered is whether academic success in a follow - up course is positively related to success attained in the pre - requisite course. The second one is whether or not the selected curriculum development approach for deciding the chains has a [UNK] impact on the academic success relationships between a pre -

	Document Source: 73 Analyzing Students Academic Success (1).pdf
	Document Source Type: Journal Paper
	Document Distance: 1.5172581672668457


## YES! WE DID IT!

# SUMMARY

WHY WE NEED A PERSISTENT CHROMADB?
In the context of a Retrieval-Augmented Generation (RAG) approach, saving and loading a persistent ChromaDB is particularly important for several reasons:

Enhanced Data Durability:

Importance: Ensures the retrieval database used for augmenting generative models is not lost between sessions or system restarts.
RAG Relevance: Maintains a consistent and reliable knowledge base that the generative model can reference, leading to more accurate and relevant responses.
Operational Continuity:

Importance: Allows seamless continuation of operations without needing to re-index or re-import data, saving time and computational resources.
RAG Relevance: Ensures that the generative model has continuous access to the same set of documents, which is essential for generating consistent and coherent responses over time.
Facilitating Collaboration:

Importance: Enables multiple users or systems to share and access the same dataset.
RAG Relevance: Supports collaborative development and usage of the RAG system, allowing different teams to work on improving the retrieval and generation processes simultaneously.
Scalability:

Importance: Provides a stable and persistent backend, enabling efficient handling of large datasets.
RAG Relevance: Essential for scaling the RAG system to handle more extensive and diverse knowledge bases, ensuring that the system can manage increased loads and deliver prompt, relevant information.
In a RAG system, the retriever (like ChromaDB) provides the generative model with relevant context from a knowledge base to generate informed and accurate responses. Persistent storage ensures that this knowledge base is durable, continuously available, and scalable, which is critical for the reliability, consistency, and performance of the RAG system.

.

.

.

In [ ]:
def generateAnswer(query,n_results=5):
    retrieved_documents=retrieveDocs(query,n_results)

    print("------- retreived documents -------\n")
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print(f"\tDocument Text: {doc}")
    print("------- RAG answer -------\n")
    output = chat.send_message( "QUESTION: "+ query + "\n EXCERPTS: "+ "\n".join(retrieved_documents))
    to_markdown(output.text)
    print('\n')
    return output

In [ ]:
queries =["Who are the authors suggested a new attention mechanism?",
          "Who are the authors suggested a new controllable text generation mechanism?",
          "Who is Murat Karakaya?",
          "Why do we need to control how the text is produced? ",
          "How can we use the self attention mechanism to control the text generation?",
          "Summarize the paper named Controllable Text Generation",
          "How many blocks are suggested in the transformer?",
          "What about decoder?"

    ]


In [ ]:
reply=generateAnswer(queries[0],10)

In [ ]:
to_markdown(reply.text)

In [ ]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))


In [ ]:
model.count_tokens(chat.history)

In [ ]:
response = chat.send_message(prompt)
to_markdown(response.text)


In [ ]:
import os
import openai
from openai import OpenAI

'''
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
openai_client = OpenAI()
'''
openai_client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo-1106"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "As an attentive and supportive academic assistant, "
            "your task is to provide assistance based solely on the provided"
            " excerpts. Answer the following questions, ensuring your responses"
            " are derived exclusively from the provided partial texts. "
            "If the answer cannot be found within the provided excerpts, "
            "kindly respond with 'I don't know'."
            "After answering each question, please provide a detailed "
            "explanation, breaking down the answer step by step and relating "
            "it to the provided excerpts."
            "Return your response as a Json object with two key fields: "
            " 'Answer', which should contain the value of the answer, and "
            " 'Reason', which should provide an explanation of why this answer "
            "was generated."

        },
        {"role": "user", "content": f"Question: {query}. \n Excerpts: {information}"}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
def generateAnswer(query,n_results=5):
    retrieved_documents=retrieveDocs(query,n_results)

    print("------- retreived documents -------\n")
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print(f"\tDocument Text: {doc}")
    print("------- RAG answer -------\n")
    output = rag(query=query, retrieved_documents=retrieved_documents)
    print(output)
    print('\n')
    return output

In [ ]:
reply=generateAnswer(queries[5],10)


In [ ]:
# prompt: convert the 'reply' to a dict

import ast
reply_dict = ast.literal_eval(reply)
print(f"Answer: {reply_dict['Answer']}")
print(f"Because; {reply_dict['Reason']}")

In [ ]:
for query in queries:
  generateAnswer(query)

In [ ]:
%pip install umap-learn

In [ ]:
def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings),2))
    for i, embedding in enumerate(tqdm(embeddings)):
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings

In [ ]:
import umap.umap_ as umap

embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10)
plt.gca().set_aspect('equal', 'datalim')
plt.title('Projected Embeddings')
plt.axis('off')

In [ ]:
query = queries[3]

results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(document)
    print('')


In [ ]:
query_embedding = embedding_function([query])[0]
retrieved_embeddings = results['embeddings'][0]

projected_query_embedding = project_embeddings([query_embedding], umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)


In [ ]:
# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_query_embedding[:, 0], projected_query_embedding[:, 1], s=150, marker='X', color='r')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none', edgecolors='g')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{query}')
plt.axis('off')

In [ ]:
def augment_query_generated(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "Sen TÜBİTAK proje başvurularını inceleyen yapay zeka konusunda uzman bir akasemisyensin."
            "Aşağıda verilen soruya, aşağıdaki proje tanımına uygun olabilecek bir cevap üret: \n"
            "Projenin genel amacı, bankacılık sektöründeki risk yönetimi operasyonlarını geliştirmek ve finansal kurumların karşılaştığı zorlukları ele almak "
            "için yapay zeka (AI) tabanlı bir platform geliştirmektir. Proje, bankalara vadeli mevduatın erken bozulması, kredilerin erken ödenmesi ve çeşitli "
            "mevduat türlerinin belirlenmesi gibi davranışsal riskleri daha etkili bir şekilde yönetme kapasitesi sunmayı hedeflemektedir. Bu riskler, finansal "
            "kurumların bilanço dengesini etkileyebilir ve operasyonel verimliliği azaltabilir. "
            "Projenin çözmeyi amaçladığı temel problem, bankaların karlılık ve risk analizlerini gerçekleştirirken karşılaştığı karmaşık durumları doğru ve "
            "etkili bir şekilde yönetme ihtiyacıdır. Özellikle vadeli mevduatların erken kapanması ve kredilerin erken ödenmesi gibi durumlar, bankaların "
            "gelecekteki nakit akışlarını ve risk profillerini belirleme sürecini karmaşıklaştırabilir"

        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
original_query = queries[0]
hypothetical_answer = augment_query_generated(original_query)

joint_query = f"{original_query} {hypothetical_answer}"
print(joint_query)

In [ ]:
def extend_retrieved_documents(results, extension=4):
  original_ids= results['ids'][0]
  print("original_ids: ",original_ids)

  extended_ids = set()


  for id in original_ids:
    extended_ids.add(int(id))
    for i in range(1, extension):
      extended_ids.add(int(id)+i)


  extended_ids = sorted([int(x) for x in extended_ids])
  extended_ids = [str(x) for x in extended_ids if int(x) < chroma_collection.count()]
  print("extended_ids: ",extended_ids)
  return chroma_collection.get(extended_ids)['documents']

In [ ]:
def retrieveDocs_augmented_query(query, n_results=5, extension=4):
    hypothetical_answer = augment_query_generated(query)
    print("------ hypothetical_answer ---------\n")
    print(hypothetical_answer,"\n")
    print("------------------------------------\n")
    joint_query = f"{query} {hypothetical_answer}"
    results = chroma_collection.query(query_texts=joint_query, n_results=n_results, include=['documents', 'embeddings'])
    retrieved_documents = extend_retrieved_documents(results, extension)
    #retrieved_documents = results['documents'][0]

    return retrieved_documents



In [ ]:
retrieved_documents=retrieveDocs_augmented_query(query, 5)

for doc in retrieved_documents:
    print(doc)
    print('')

In [ ]:
results = chroma_collection.query(query_texts=joint_query, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]

for doc in retrieved_documents:
    print(doc)
    print('')

In [ ]:
retrieved_embeddings = results['embeddings'][0]
original_query_embedding = embedding_function([original_query])
augmented_query_embedding = embedding_function([joint_query])

projected_original_query_embedding = project_embeddings(original_query_embedding, umap_transform)
projected_augmented_query_embedding = project_embeddings(augmented_query_embedding, umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none', edgecolors='g')
plt.scatter(projected_original_query_embedding[:, 0], projected_original_query_embedding[:, 1], s=150, marker='X', color='r')
plt.scatter(projected_augmented_query_embedding[:, 0], projected_augmented_query_embedding[:, 1], s=150, marker='X', color='orange')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{original_query}')
plt.axis('off')

In [ ]:
def generateAnswer_augmented_query(query,n_results=5, extention=4):
    print("------- query -------\n")
    print(query,"\n")
    retrieved_documents=retrieveDocs_augmented_query(query,n_results,extention)
    print("------- retreived documents -------\n")
    for document in retrieved_documents:
        print(document)
        print('\n')

    print("------- RAG answer -------\n")
    output = rag(query=query, retrieved_documents=retrieved_documents)
    print(output)
    print('\n')

In [ ]:
queries

In [ ]:
generateAnswer_augmented_query(queries[0],10,5)

In [ ]:
title= """Ar-Ge Sürecinde Kullanılacak Yöntemler Tanımlanan proje hedeflerine ulaşmak için uygulanacak analitik
        deneysel çözüm yöntemlerini belirtiniz. (NOT: Bu bölümde sunulan proje özelinde
        hangi teknik / bilimsel yaklaşımların ve bunlara ait aşamaların takip edileceği açıklanmalı, iş paketleri isimleri ya da her projede olabilecek standart
        rutin çalışma yöntemleri tekrarlanmamalıdır."""
results = chroma_collection.query(query_texts=title, n_results=5, include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]
print(retrieved_documents)

In [ ]:
title= """Ar-Ge Sürecinde Kullanılacak Yöntemler Tanımlanan proje hedeflerine ulaşmak için uygulanacak analitik
        deneysel çözüm yöntemlerini belirtiniz. (NOT: Bu bölümde sunulan proje özelinde
        hangi teknik / bilimsel yaklaşımların ve bunlara ait aşamaların takip edileceği açıklanmalı, iş paketleri isimleri ya da her projede olabilecek standart
        rutin çalışma yöntemleri tekrarlanmamalıdır."""
results = chroma_collection.query(query_texts=title, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = extend_retrieved_documents(results)
print(retrieved_documents)


In [ ]:
chroma_collection.get(results['ids'][0])